In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import networkx as nx


In [2]:
adjacencytable_transformed = pd.read_csv(r"C:\Users\EvanSpiller\Documents\DS BootCamp\Capstone\data\adjacency_csv.csv")
summary_by_quarter_rideshares = pd.read_csv(r"C:\Users\EvanSpiller\Documents\DS BootCamp\Capstone\data\summary_by_quarter_rideshares.csv")
summary_by_quarter_rideshares['Year-Quarter'] = summary_by_quarter_rideshares['Year'].astype(str) + '-Q' + summary_by_quarter_rideshares['Quarter'].astype(str)
summary_by_quarter_rideshares.value_counts('Year-Quarter')
merged_df = pd.merge(summary_by_quarter_rideshares, adjacencytable_transformed, left_on=['PULocationID','DOLocationID'], right_on=['LocID', 'Adjacent_District'])
merged_df = merged_df.drop(['LocID','Adjacent_District'], axis =1)

In [40]:
merged_df.columns

Index(['Year', 'Quarter', 'DOLocationID', 'PULocationID', 'trips', 'cost',
       'Year-Quarter'],
      dtype='object')

In [3]:
def best_route_calculator(start, end, zones, weight_type):
    G = nx.DiGraph()
    for _, row in df_filtered.iterrows():
        G.add_edge(row['PULocationID'], row['DOLocationID'], weight=row[weight_type])
    #want to retry this while maximizing for dollars spent
    #this needs to be on the original dataframe
    result_dict = {}

    for loc, adj_loc, dist in zip(df_filtered['DOLocationID'], df_filtered['PULocationID'], df_filtered[weight_type]):
        key = (loc, adj_loc)  # Combine loc and adj_loc into a tuple
        result_dict[key] = dist

    paths = []
    result_list = []
    path_combo_list = []


    for path in nx.all_simple_paths(G, source=start, target=end, cutoff = zones):
        paths.append(path)

    paths_DF = pd.DataFrame({'paths':paths})

    for path in paths_DF['paths']:
        path_combinations = []
        for i in range(len(path)):
            for j in range(i + 1, len(path)):
                path_combinations.append((path[i],path[j]))
        result_list.append(path_combinations)

    # Print the result_list or perform further processing
    for path_combinations in result_list:
        path_combo_list.append(path_combinations)


    sums = []
    for l in range(len(path_combo_list)):
        list1 = []
        for val in range(len(path_combo_list[l])):
            try:
                list1.append(result_dict[path_combo_list[l][val]])
            except KeyError:
                list1.append(0)  # Append 0 in case of key error
        sumnums = sum(list1)
        sums.append(sumnums)

    pathdf = pd.DataFrame({'paths':paths, 'path_combo_list': path_combo_list, 'sums': sums})

    top  = pathdf.sort_values('sums', ascending=False).head(1)
    weights_column = top['paths']
    toplist = weights_column.tolist()

    longestpath = pd.DataFrame({'Order': range(len(toplist[0])), 'PathIDs': toplist[0], 'Weight_of_path': top['sums'].item() })
    return longestpath

In [25]:
Shortest = nx.DiGraph()
for _, row in adjacencytable_transformed.iterrows():
    Shortest.add_edge(row['LocID'], row['Adjacent_District'])
print(len(nx.shortest_path(Shortest, source=14, target=150)))
print(len(nx.shortest_path(Shortest, source=14, target=150))*1.5)

7
10.5


In [27]:
Year_quarters = ['2017-Q3', '2017-Q4', '2018-Q1', '2018-Q2', '2018-Q3',
       '2018-Q4', '2019-Q1', '2019-Q2', '2019-Q3', '2019-Q4', '2020-Q1',
       '2020-Q2', '2020-Q3', '2020-Q4', '2021-Q1', '2021-Q2', '2021-Q3',
       '2021-Q4', '2022-Q1', '2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1']

In [34]:
trips = 'trips'
best_routes = []
for q in Year_quarters:
    df_filtered = merged_df.loc[merged_df['Year-Quarter']==q]
    best_route = best_route_calculator(14, 150, 11, trips)
    best_route['Quarter'] = q
    best_routes.append(best_route)

In [38]:
best_routes

,Order,PathIDs,Quarter
0,0,14.0,2017-Q3
1,1,228.0,2017-Q3
2,2,181.0,2017-Q3
3,3,189.0,2017-Q3
4,4,61.0,2017-Q3
...,...,...,...
271,7,72.0,2023-Q1
272,8,39.0,2023-Q1
273,9,155.0,2023-Q1
274,10,154.0,2023-Q1


In [39]:
best_routes.to_csv(r'C:\Users\EvanSpiller\Documents\DS BootCamp\Capstone\data\14_to_150_byquarter.csv', index=False)

In [130]:
Year_quarters = ['2019-Q1', '2019-Q2', '2019-Q3', '2019-Q4', '2020-Q1',
       '2020-Q2', '2020-Q3', '2020-Q4', '2021-Q1', '2021-Q2', '2021-Q3',
       '2021-Q4', '2022-Q1', '2022-Q2', '2022-Q3', '2022-Q4']
#with trip cost

In [107]:
cost = 'cost'
best_routes = []
for q in Year_quarters:
    df_filtered = merged_df.loc[merged_df['Year-Quarter']==q]
    best_route = best_route_calculator(14, 150, 11, cost)
    best_route['Quarter'] = q
    best_routes.append(best_route)

In [110]:
best_routes = pd.concat(best_routes, ignore_index=True)

In [111]:
best_routes

,Order,PathIDs,Weight_of_path,Quarter
0,0,14.0,1709524.77,2019-Q1
1,1,228.0,1709524.77,2019-Q1
2,2,181.0,1709524.77,2019-Q1
3,3,189.0,1709524.77,2019-Q1
4,4,61.0,1709524.77,2019-Q1
...,...,...,...,...
187,7,72.0,2794873.22,2022-Q4
188,8,39.0,2794873.22,2022-Q4
189,9,155.0,2794873.22,2022-Q4
190,10,154.0,2794873.22,2022-Q4


In [112]:
best_routes.to_csv(r'C:\Users\EvanSpiller\Documents\DS BootCamp\Capstone\data\14_to_150_byquarter_weightedbycost.csv', index=False)

Let's run this algorithms for 15 proposed bus routes.

In [20]:
Routes = ['Bay_Ridge_to_Manhattan_Beach',
'Bay_Ridge_to_Sheepshead_Bay',
'Gravesend_to_Canarsie',
'Midwood_to_Sunset_Park',
'Downtown_Brooklyn_to_Ridgewood',
'Williamsburg_to_Lower_East_Side',
'Prospect_Lefferts_Garden_to_Bergen_Beach',
'Downtown_Brooklyn_to_Kings_Plaza',
'Greenpoint_to_Prospect_Lefferts_Garden',
'Bed_Stuy_to_Sheepshead_Bay',
'Flatbush_to_Downtown_BK',
'Broadway_Junction_to_Sunnyside',
'Downtown_Brooklyn_to_Jackson_Heights',
'East_New_York_to_Midwood',
'Williamsburg_to_Jamaica']

Starts = [14,14,108,165,65,
256,
188,
65,
112,
17,
89,
177,
65,
165,
217]

Ends = [150,210,39,228,198,
148,
155,
155,
188,
210,
65,
226,
129,
76,
130]

Route_creator_2 = [Routes,Starts,Ends, []]

In [ ]:
Shortest = nx.DiGraph()
for _, row in adjacencytable_transformed.iterrows():
    Shortest.add_edge(row['LocID'], row['Adjacent_District'])

In [21]:
for x in range(len(Route_creator_2[0])):
    stops = len(nx.shortest_path(Shortest, source=Route_creator_2[1][x], target=Route_creator_2[2][x]))*1.5
    Route_creator_2[3].append(stops)

In [11]:
Year_quarters = ['2017-Q3', '2017-Q4', '2018-Q1', '2018-Q2', '2018-Q3',
       '2018-Q4', '2019-Q1', '2019-Q2', '2019-Q3', '2019-Q4', '2020-Q1',
       '2020-Q2', '2020-Q3', '2020-Q4', '2021-Q1', '2021-Q2', '2021-Q3',
       '2021-Q4', '2022-Q1', '2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1']

In [24]:
trips = 'trips'
best_routes = []
for x in range(len(Route_creator_2[0])):
    for q in Year_quarters:
        df_filtered = merged_df.loc[merged_df['Year-Quarter']==q]
        best_route = best_route_calculator(Route_creator_2[1][x], Route_creator_2[2][x], Route_creator_2[3][x], trips)
        best_route['Quarter'] = q
        best_route['Route'] = Route_creator_2[0][x]
        best_routes.append(best_route)

In [35]:
best_routes = pd.concat(best_routes, ignore_index=True)

In [36]:
best_routes

,Order,PathIDs,Weight_of_path,Quarter,Route
0,0,14.0,1709524.77,2019-Q1,Bay_Ridge_to_Manhattan_Beach
1,1,228.0,1709524.77,2019-Q1,Bay_Ridge_to_Manhattan_Beach
2,2,181.0,1709524.77,2019-Q1,Bay_Ridge_to_Manhattan_Beach
3,3,189.0,1709524.77,2019-Q1,Bay_Ridge_to_Manhattan_Beach
4,4,61.0,1709524.77,2019-Q1,Bay_Ridge_to_Manhattan_Beach
5,5,35.0,1709524.77,2019-Q1,Bay_Ridge_to_Manhattan_Beach
6,6,76.0,1709524.77,2019-Q1,Bay_Ridge_to_Manhattan_Beach
7,7,72.0,1709524.77,2019-Q1,Bay_Ridge_to_Manhattan_Beach
8,8,39.0,1709524.77,2019-Q1,Bay_Ridge_to_Manhattan_Beach
9,9,155.0,1709524.77,2019-Q1,Bay_Ridge_to_Manhattan_Beach


In [33]:
best_routes.to_csv(r'C:\Users\EvanSpiller\Documents\DS BootCamp\Capstone\data\15routes_byquarter_bytrips.csv', index=False)

AttributeError: 'list' object has no attribute 'to_csv'

Running by cost now

In [37]:
Year_quarters = ['2019-Q1', '2019-Q2', '2019-Q3', '2019-Q4', '2020-Q1',
       '2020-Q2', '2020-Q3', '2020-Q4', '2021-Q1', '2021-Q2', '2021-Q3',
       '2021-Q4', '2022-Q1', '2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1']

In [38]:
cost = 'cost'
best_routes = []
for x in range(len(Route_creator_2[0])):
    for q in Year_quarters:
        df_filtered = merged_df.loc[merged_df['Year-Quarter']==q]
        best_route = best_route_calculator(Route_creator_2[1][x], Route_creator_2[2][x], Route_creator_2[3][x], cost)
        best_route['Quarter'] = q
        best_route['Route'] = Route_creator_2[0][x]
        best_routes.append(best_route)

In [39]:
best_routes = pd.concat(best_routes, ignore_index=True)
best_routes

,Order,PathIDs,Weight_of_path,Quarter,Route
0,0,14.0,1709524.77,2019-Q1,Bay_Ridge_to_Manhattan_Beach
1,1,228.0,1709524.77,2019-Q1,Bay_Ridge_to_Manhattan_Beach
2,2,181.0,1709524.77,2019-Q1,Bay_Ridge_to_Manhattan_Beach
3,3,189.0,1709524.77,2019-Q1,Bay_Ridge_to_Manhattan_Beach
4,4,61.0,1709524.77,2019-Q1,Bay_Ridge_to_Manhattan_Beach
...,...,...,...,...,...
2409,5,76.0,2985297.67,2023-Q1,Williamsburg_to_Jamaica
2410,6,124.0,2985297.67,2023-Q1,Williamsburg_to_Jamaica
2411,7,132.0,2985297.67,2023-Q1,Williamsburg_to_Jamaica
2412,8,216.0,2985297.67,2023-Q1,Williamsburg_to_Jamaica


In [40]:
best_routes.to_csv(r'C:\Users\EvanSpiller\Documents\DS BootCamp\Capstone\data\15routes_byquarter_bycost.csv', index=False)